In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

california = fetch_california_housing()

df = pd.DataFrame(california.data, columns = california.feature_names)
df["Target"] = california.target
df.tail()

# 데이터 정규화
scaler = StandardScaler()
scaler.fit(df.values[:,:-1])
df.iloc[:,:-1] = scaler.transform(df.values[:,:-1]).round(4)

In [2]:
# 학습코드 구현
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

data = torch.from_numpy(df.values).float()

x = data[:,:-1]
y = data[:,-1:]

print(x.shape, y.shape)

torch.Size([20640, 8]) torch.Size([20640, 1])


In [3]:
# 하이퍼파라미터 설정
n_epochs = 4000
batch_size = 256 # 미니배치 크기(한번에 처리하는 데이터 샘플 개수)
print_interval = 200
# learning_rate = 1e-2 ---> 학습률 필요 없음(초기에 큰 학습률을 가지면서 점차 감쇠 + 매번 계산된 그래디언트를 누적하는 형태구현 : 아담)

In [4]:
# 심층신경망 정의 (너비 : 6, 깊이 : 5)
model = nn.Sequential(
    nn.Linear(x.size(-1),6),
    nn.LeakyReLU(),
    nn.Linear(6,5),
    nn.LeakyReLU(),
    nn.Linear(5,4),
    nn.LeakyReLU(),
    nn.Linear(4,3),
    nn.LeakyReLU(),
    nn.Linear(3,y.size(-1)),
)

In [6]:
optimizer = optim.Adam(model.parameters()) # 인자에 학습률은 넣지 않음 (각 파라미터마다 학습률을 자동으로 조정하기 때문, 기본 학습률 : 1e-2)

In [7]:
import time
start = time.time()
for i in range(n_epochs):
    indices = torch.randperm(x.size(0))
    x_ = torch.index_select(x, dim=0, index=indices)
    y_ = torch.index_select(y, dim=0, index=indices)

    x_ = x_.split(batch_size, dim=0)
    y_ = y_.split(batch_size, dim=0)

    y_hat = []
    total_loss = 0

    for x_i, y_i in zip(x_,y_):
        y_hat_i = model(x_i)
        loss = F.mse_loss(y_hat_i, y_i)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        total_loss += float(loss) # 메모리 손실을 없애기 위해

        y_hat += [y_hat_i]

    total_loss = total_loss / len(x_)
    if(i+1) % print_interval == 0:
        print('Epoch %d: loss=%.4e' % (i+1, total_loss))

y_hat = torch.cat(y_hat, dim=0)
y = torch.cat(y_, dim=0)

end = time.time()
elapsed = end - start
print('Elapsed time: %dm %ds'%(elapsed//60, int(elapsed - elapsed//60)))

/var/folders/dc/3dqm8vys2qv2c52_q3sn76xr0000gn/T/ipykernel_11756/2139505422.py:23: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  total_loss += float(loss) # 메모리 손실을 없애기 위해


Epoch 200: loss=3.1970e-01
Epoch 400: loss=3.1041e-01
Epoch 600: loss=3.0580e-01
Epoch 800: loss=2.9977e-01
Epoch 1000: loss=2.9722e-01
Epoch 1200: loss=2.9475e-01
Epoch 1400: loss=2.9307e-01
Epoch 1600: loss=2.9338e-01
Epoch 1800: loss=2.9256e-01
Epoch 2000: loss=2.9235e-01
Epoch 2200: loss=2.9245e-01
Epoch 2400: loss=2.9170e-01
Epoch 2600: loss=2.9293e-01
Epoch 2800: loss=2.9206e-01
Epoch 3000: loss=2.9205e-01
Epoch 3200: loss=2.9118e-01
Epoch 3400: loss=2.9199e-01
Epoch 3600: loss=2.9288e-01
Epoch 3800: loss=2.9096e-01
Epoch 4000: loss=2.9124e-01
Elapsed time: 1m 66s
